<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [ ]:
import pandas as pd
import csv

In [ ]:
with open('data/chipotle.tsv', 'r') as csvfile:
    chipotle_data = csv.reader(csvfile, delimiter = '\t')
    file_nested_list = [row for row in chipotle_data]

In [ ]:
file_nested_list

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [ ]:
# grab first list (column names/header)
header = file_nested_list[0]
# grab everything except the first list
data = file_nested_list[1:]

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [ ]:
# Moving the data into pandas because it's easier to work with :)
data = pd.DataFrame(data=data, columns=header)
data.head()

In [ ]:
# add an additional column that converts the item_price string into a float for easier calculation
data['price_numeric'] = [float(x.replace('$', '')) for x in data.item_price]
# convert quantity column to numeric dtype instead of object dtype
data.quantity = pd.to_numeric(data.quantity)
data.dtypes

In [ ]:
"""
Since the item_price column is ACTUALLY the order_price (not the item unit price), we do not need to take quantity into
account when calculating the average order price.  Instead, we can just consider the mean value
of all order prices (e.g. the mean of the item_price column).  I am calculating the item unit_price
for each order and storing that into the unit_price column in case it is needed.
"""
data['unit_price'] = data.price_numeric/data.quantity
print 'The average order cost is: ', data.price_numeric.mean()

### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [ ]:
# Store a copy of the original data subsetting where the item name is 'Canned Soda' or 'Canned Soft Drink'
soda_orders = data[(data.item_name == 'Canned Soda') | (data.item_name == 'Canned Soft Drink')].copy()

In [ ]:
# Store unique soda names into list
unique_sodas = list(pd.unique(soda_orders.choice_description))

In [ ]:
unique_sodas

---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [ ]:
# Subset order data into only orders that contain 'burrito' in the name and store a copy
# str.lower() is included to simplify the contains method
burrito_orders = data[(data.item_name.str.lower().str.contains('burrito'))].copy()

In [ ]:
burrito_orders.head()

In [ ]:
# split choice_description on commas to create lists where each element is an individual topping
# Store the length of this list as the number of toppings for each order
burrito_orders['num_toppings'] = [len(x.split(',')) for x in burrito_orders.choice_description]
# Calculate the average number of toppings per burrito using .mean()
print "The average number of toppings per burrito is: ", burrito_orders.num_toppings.mean()

### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [ ]:
# Store a copy of all data that contains 'chip' in the name
chip_orders = data[(data.item_name.str.lower().str.contains('chip'))].copy()
# Create a dictionary of chip orders grouped by name with values as quantity sum
chip_counts = dict(chip_orders.groupby('item_name').quantity.sum())
chip_counts
# Currently, this does not consider 'Side of chips' and 'Chips' as the same thing. These are treated as distinct orders

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!
